# Tworzymy nazwane klastry

W lekcji tej użyjemy LLM-ów do pomocy przy znalezieniu odpowiednich nazw i opisów dla klastrów.

In [1]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import dotenv_values
from pycaret.clustering import predict_model, load_model

In [1]:
import pycaret
print(pycaret.__version__)


3.3.2


In [2]:
env = dotenv_values(".env")

openai_client = OpenAI(api_key=env["OPENAI_API_KEY"])

ładujemy nasze dane

In [3]:
df = pd.read_csv('welcome_survey_simple_v2.csv', sep=';')
df.head()

,age,edu_level,fav_animals,fav_place,gender
0,<18,Podstawowe,Brak ulubionych,NaN,Kobieta
1,25-34,Średnie,Psy,Nad wodą,Mężczyzna
2,45-54,Wyższe,Psy,W lesie,Mężczyzna
3,35-44,Średnie,Koty,W górach,Mężczyzna
4,35-44,Wyższe,Psy,Nad wodą,Mężczyzna


ładujemy model wytrenowany w poprzedniej lekcji

In [5]:
kmeans_pipeline = load_model('welcome_survey_clustering_pipeline_v1_model_szczepan_v1')

Transformation Pipeline and Model Successfully Loaded


aplikujemy model do danych

In [6]:
df_with_clusters = predict_model(model=kmeans_pipeline, data=df)
df_with_clusters["Cluster"].value_counts()

Cluster 0    50
Cluster 1    34
Cluster 6    28
Cluster 2    23
Cluster 3    21
Cluster 4    18
Cluster 7    17
Cluster 8    16
Cluster 5    13
Cluster 9     9
Name: Cluster, dtype: int64

stworzymy teraz prompt, który prześlemy do LLM-a w celu znalezienia odpowiednich nazw i opisów dla klastrów

In [7]:
cluster_descriptions = {}
for cluster_id in df_with_clusters['Cluster'].unique():
    cluster_df = df_with_clusters[df_with_clusters['Cluster'] == cluster_id]
    summary = ""
    for column in df_with_clusters:
        if column == 'Cluster':
            continue

        value_counts = cluster_df[column].value_counts()
        value_counts_str = ', '.join([f"{idx}: {cnt}" for idx, cnt in value_counts.items()])
        summary += f"{column} - {value_counts_str}\n"

    cluster_descriptions[cluster_id] = summary

In [8]:
print(cluster_descriptions["Cluster 5"])

age - 35-44: 7, >=65: 2, 45-54: 1, 55-64: 1, <18: 1, unknown: 1, 18-24: 0, 25-34: 0
edu_level - Wyższe: 12, Podstawowe: 1, Średnie: 0
fav_animals - Brak ulubionych: 13, Inne: 0, Koty: 0, Koty i Psy: 0, Psy: 0
fav_place - Nad wodą: 5, W lesie: 4, Inne: 0, W górach: 0
gender - Mężczyzna: 8, Kobieta: 4



In [9]:
prompt = "Użyliśmy algorytmu klastrowania."
for cluster_id, description in cluster_descriptions.items():
    prompt += f"\n\nKlaster {cluster_id}:\n{description}"

prompt += """
Wygeneruj najlepsze nazwy dla każdego z klasterów oraz ich opisy

Użyj formatu JSON. Przykładowo:
{
    "Cluster 0": {
        "name": "Klaster 0",
        "description": "W tym klastrze znajdują się osoby, które..."
    },
    "Cluster 1": {
        "name": "Klaster 1",
        "description": "W tym klastrze znajdują się osoby, które..."
    }
}
"""
print(prompt)

Użyliśmy algorytmu klastrowania.

Klaster Cluster 5:
age - 35-44: 7, >=65: 2, 45-54: 1, 55-64: 1, <18: 1, unknown: 1, 18-24: 0, 25-34: 0
edu_level - Wyższe: 12, Podstawowe: 1, Średnie: 0
fav_animals - Brak ulubionych: 13, Inne: 0, Koty: 0, Koty i Psy: 0, Psy: 0
fav_place - Nad wodą: 5, W lesie: 4, Inne: 0, W górach: 0
gender - Mężczyzna: 8, Kobieta: 4


Klaster Cluster 7:
age - 35-44: 8, 45-54: 4, 18-24: 2, 25-34: 2, >=65: 1, 55-64: 0, <18: 0, unknown: 0
edu_level - Średnie: 17, Podstawowe: 0, Wyższe: 0
fav_animals - Psy: 13, Koty: 2, Brak ulubionych: 1, Inne: 1, Koty i Psy: 0
fav_place - Nad wodą: 12, W lesie: 2, Inne: 0, W górach: 0
gender - Mężczyzna: 12, Kobieta: 5


Klaster Cluster 3:
age - 45-54: 20, 55-64: 1, 18-24: 0, 25-34: 0, 35-44: 0, <18: 0, >=65: 0, unknown: 0
edu_level - Wyższe: 21, Podstawowe: 0, Średnie: 0
fav_animals - Psy: 11, Koty: 5, Inne: 4, Brak ulubionych: 1, Koty i Psy: 0
fav_place - W lesie: 19, Inne: 2, Nad wodą: 0, W górach: 0
gender - Mężczyzna: 17, Kobieta:

In [10]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
)

In [21]:
result = response.choices[0].message.content.replace("```json", "").replace("```", "").strip()
cluster_names_and_descriptions_szv1 = json.loads(result)

In [22]:
with open("welcome_survey_cluster_names_and_descriptions_szv1.json", "w") as f:
    f.write(json.dumps(cluster_names_and_descriptions))

In [23]:
with open("welcome_survey_cluster_names_and_descriptions_szv1.json", "r") as f:
    print(json.loads(f.read()))

{'Cluster 0': {'name': 'Wodni Miłośnicy Psów z Wyższym Wykształceniem', 'description': 'W tym klastrze znajdują się osoby, które preferują spędzanie czasu nad wodą, mają wyższe wykształcenie i najczęściej wybierają psy jako ulubione zwierzęta. Większość z nich to mężczyźni w wieku 35-44 lat.'}, 'Cluster 1': {'name': 'Górscy Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują góry jako miejsce wypoczynku. Większość z nich to mężczyźni w wieku 45-54 lat, a ich ulubionymi zwierzętami są psy.'}, 'Cluster 2': {'name': 'Koci Miłośnicy Gór', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują góry i mają skłonność do wybierania kotów jako ulubionych zwierząt. Większość z nich to mężczyźni w wieku 35-44 lat.'}, 'Cluster 3': {'name': 'Leśni Profesjonaliści', 'description': 'W tym klastrze znajdują się osoby z wyższym wykształceniem, które preferują lasy jako miejsce wypoczynku. Większość z nich to 